# Trabalho Final

## Coleta Preparação e Análise de Dados

#### Aléxia de Jesus Dorneles Pereira, Arthur Land Avila, Gustavo Giongo Lottermann, Matheus Gonzaga Krebs e Vicente Hofmeister

Trabalho feito utilizando [biblioteca Pysus](https://github.com/AlertaDengue/PySUS/tree/main). Caso necessário, confira a [documentação](https://pysus.readthedocs.io/en/latest/).

## Pysus

In [1]:
import os
import pandas as pd
from pysus.online_data import SINAN as SINAN_Online
from pysus.online_data import SINASC as SINASC_Online
from pysus.online_data import SIM as SIM_ONLINE
from pysus.ftp.databases.sinan import SINAN

Import de bibliotecas

A sessão de código a seguir salva em listas os anos que os dados de sinan, sinasc e sim estão disponiveis

In [2]:
states = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', \
          'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', \
          'RR', 'SC', 'SP', 'SE', 'TO']

sinan_years = SINAN_Online.get_available_years("DENG")
sinasc_years = SINASC_Online.get_available_years(group= 'DN', states = states)
sim_years = SIM_ONLINE.get_available_years(group= 'CID10', states = states)

Confere os anos em que todas bases de dados estão disponiveis simultaneamente

In [3]:
avaliable_years = []

for year in sinan_years:
      if year in sinasc_years and year in sim_years:
            avaliable_years.append(year)

busca arquivos SINAN baseado nos anos definidos em years, descreve-os e realiza download dos datasets

In [4]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
sinan = SINAN().load()
sinan_files = sinan.get_files(dis_code=['DENG'], year=years)

for file in sinan_files:
      print(sinan.describe(file))

parquet_path = os.getcwd() + "/Data/Parquet/"
# sinan.download(sinan_files, local_dir=parquet_path)

{'name': 'DENGBR12.dbc', 'disease': 'Dengue', 'year': 2012, 'size': '31.3 MB', 'last_update': '2022-02-22 10:28AM'}
{'name': 'DENGBR13.dbc', 'disease': 'Dengue', 'year': 2013, 'size': '65.5 MB', 'last_update': '2022-02-18 10:03AM'}
{'name': 'DENGBR14.dbc', 'disease': 'Dengue', 'year': 2014, 'size': '42.7 MB', 'last_update': '2021-11-23 12:15PM'}
{'name': 'DENGBR15.dbc', 'disease': 'Dengue', 'year': 2015, 'size': '109.2 MB', 'last_update': '2021-11-23 12:15PM'}
{'name': 'DENGBR16.dbc', 'disease': 'Dengue', 'year': 2016, 'size': '99.4 MB', 'last_update': '2021-11-23 12:16PM'}
{'name': 'DENGBR17.dbc', 'disease': 'Dengue', 'year': 2017, 'size': '22.9 MB', 'last_update': '2021-11-23 12:16PM'}
{'name': 'DENGBR18.dbc', 'disease': 'Dengue', 'year': 2018, 'size': '21.3 MB', 'last_update': '2021-11-23 12:16PM'}
{'name': 'DENGBR19.dbc', 'disease': 'Dengue', 'year': 2019, 'size': '110.2 MB', 'last_update': '2021-11-23 12:17PM'}
{'name': 'DENGBR20.dbc', 'disease': 'Dengue', 'year': 2020, 'size': '7

In [12]:
csv_path = os.getcwd() + '/Data/CSV/'
files_names = os.listdir(parquet_path)
for file in files_names:
      df = pd.read_parquet(path= parquet_path + file)
      print(file)
      df.to_csv(path_or_buf = (csv_path + file.replace('.parquet', '.csv')), sep = ';')
      
      parquet_files = os.listdir(parquet_path + file)
      for pFile in parquet_files:
            os.remove(parquet_path + file + '/' + pFile)      
      os.rmdir(parquet_path + file)


DENGBR12.parquet
DENGBR15.parquet
DENGBR20.parquet
DENGBR21.parquet
DENGBR19.parquet
DENGBR13.parquet
DENGBR16.parquet
DENGBR22.parquet
DENGBR18.parquet
DENGBR14.parquet
DENGBR17.parquet
